# CloudWatch

In [2]:
%%time
import boto3
import pandas as pd
from datetime import datetime, timedelta, timezone
import io
import json
from time import sleep
from threading import Thread

import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.huggingface.model import HuggingFaceModel

CPU times: user 738 ms, sys: 152 ms, total: 890 ms
Wall time: 1.18 s


In [3]:
##S3 prefixes
bucket_name = 'aai-540-final-data-east-1'
region_name = 'us-east-1'
session = sagemaker.Session(boto3.Session(region_name=region_name))
s3 = session.boto_session.client('s3')
s3_path = 'athena/results/'

prefix = "sagemaker/AIEmotions-ModelQualityMonitor"
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket_name}/{data_capture_prefix}"


ground_truth_upload_path = (
    f"s3://{bucket_name}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket_name}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region_name)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/datacapture
Ground truth path: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-22-09-18-47
Report path: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/reports


In [4]:
'''
# Access Queried Test Data
bucket_name = 'aai-540-final-data'
s3_test_query = 'athena/results/f4cf6706-1ba6-4538-ab86-13c8137facb5.csv'
test_data_obj = s3.get_object(Bucket=bucket_name, Key = s3_test_query)
df_test = pd.read_csv(io.BytesIO(test_data_obj['Body'].read()))
'''

In [4]:
df_test_new = pd.read_csv('test.csv')
df_test_new

emotion_categories = {
	"anger": ["anger", "annoyance", "disapproval"],
	"disgust": ["disgust"],
	"fear": ["fear", "nervousness"],
	"happy": ["joy", "amusement", "approval", "gratitude"],
	"optimistic": ["optimism", "relief", "pride", "excitement"],
	"affectionate": [ "love", "caring", "admiration",  "desire"],
	"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
	"surprise": ["surprise", "realization", "confusion", "curiosity"],
	"neutral": ["neutral"]
} 

emotion_index_to_label = {index: label for index, label in enumerate(emotion_categories)}
df_test_new['emotions'] = df_test_new['emotions'].apply(lambda x: emotion_index_to_label[x])
df_test_new

,text,emotions
0,Yeah I tried to apply but with being partially...,sadness
1,"First time, health/environmental reasons. Seco...",neutral
2,Wow. I dig that.,affectionate
3,Thank you! We’re currently shoveling ourselves...,happy
4,This is one of my favorite of [NAME] songs alt...,affectionate
...,...,...
5422,"So, the kicker... her brother played on Xbox s...",neutral
5423,It would be hilarious if it was a pasta place too,happy
5424,"Oh damn, that would have been the only correct...",sadness
5425,"At this point, LSC only exists to give sociali...",anger


In [5]:
#Retrieve Model
role = sagemaker.get_execution_role()
s3 = boto3.client('s3')
model_dir = 'models/tuned_model/'
tar_file = 'model.tar.gz'


model_name = f"AIEmotion-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_data = f's3://{bucket_name}/{model_dir}{tar_file}'
tensorflow_version = '2.6.3'
transformers_version='4.17.0'
py_version = 'py38'
huggingface_model = HuggingFaceModel(model_data=model_data,
                                     role=role,
                                     transformers_version=transformers_version,
                                     tensorflow_version=tensorflow_version,
                                     py_version=py_version,
                                     entry_point='inference.py'
                                   )

In [9]:
endpoint_name = f"AIEmotion-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
#endpoint_name = 'emotion-ai-endpoint'
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = AIEmotion-model-quality-monitor-2024-02-22-0923


INFO:sagemaker:Creating model with name: huggingface-tensorflow-inference-2024-02-22-09-23-43-455
INFO:sagemaker:Creating endpoint-config with name AIEmotion-model-quality-monitor-2024-02-22-0923
INFO:sagemaker:Creating endpoint with name AIEmotion-model-quality-monitor-2024-02-22-0923


---


KeyboardInterrupt



In [ ]:
# Predictions as baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket_name}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket_name}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

In [9]:
# Grab a sample of the dataset
df_test_subset = df_test_new.sample(500)
df_test_subset = df_test_subset.reset_index(drop=True)

In [10]:
def calculate_predictions(results):
    df_results = pd.DataFrame(columns = ['pred_labels', 'probabilities'])
    for result in results:
        res = json.loads(result[0])['slack_app']
        pred_labels = list(res.keys())
        prob = list(res.values())
        row = {'pred_labels': pred_labels, 'probabilities': prob}
        df_results = df_results.append(row, ignore_index = True)
    return df_results

In [11]:
predictions = df_test_subset.apply(lambda x: predictor.predict({'text': x['text']}), axis = 1)
predictions

0      [{"slack_app": {"neutral": 0.6640094518661499,...
1      [{"slack_app": {"neutral": 0.5344992876052856,...
2      [{"slack_app": {"anger": 0.558377742767334, "d...
3      [{"slack_app": {"surprise": 0.5107897520065308...
4      [{"slack_app": {"happy": 0.9388467669487, "ang...
                             ...                        
495    [{"slack_app": {"optimistic": 0.46600499749183...
496    [{"slack_app": {"anger": 0.8569743037223816, "...
497    [{"slack_app": {"sadness": 0.6915997266769409,...
498    [{"slack_app": {"affectionate": 0.511790454387...
499    [{"slack_app": {"happy": 0.9782002568244934, "...
Length: 500, dtype: object

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df_preds = calculate_predictions(predictions)
df_preds

,pred_labels,probabilities
0,"[neutral, anger, sadness]","[0.6640094518661499, 0.20224148035049438, 0.06..."
1,"[neutral, anger, happy]","[0.5344992876052856, 0.389532208442688, 0.0258..."
2,"[anger, disgust, neutral]","[0.558377742767334, 0.2215181142091751, 0.0868..."
3,"[surprise, neutral, anger]","[0.5107897520065308, 0.29275429248809814, 0.10..."
4,"[happy, anger, neutral]","[0.9388467669487, 0.01765925996005535, 0.01090..."
...,...,...
495,"[optimistic, neutral, happy]","[0.46600499749183655, 0.1959737241268158, 0.17..."
496,"[anger, neutral, disgust]","[0.8569743037223816, 0.07698012888431549, 0.02..."
497,"[sadness, affectionate, anger]","[0.6915997266769409, 0.08669991046190262, 0.05..."
498,"[affectionate, neutral, happy]","[0.5117904543876648, 0.3719044029712677, 0.056..."


In [13]:
# Extract the first element of each tuple
df_preds['pred_labels'] = df_preds['pred_labels'].apply(lambda x: x[0])
df_preds['probabilities'] = df_preds['probabilities'].apply(lambda x: x[0])
df_preds

,pred_labels,probabilities
0,neutral,0.664009
1,neutral,0.534499
2,anger,0.558378
3,surprise,0.510790
4,happy,0.938847
...,...,...
495,optimistic,0.466005
496,anger,0.856974
497,sadness,0.691600
498,affectionate,0.511790


In [14]:
# Concat datasets
validate_dataset = pd.concat([df_test_subset, df_preds], ignore_index=True, axis = 1)
validate_dataset = validate_dataset.rename(columns = {0: 'text', 
                                                      1: 'true_label', 
                                                      2: 'pred_label', 
                                                      3: 'probability'})

validate_dataset['pred_label'] = validate_dataset['pred_label'].replace({'sad': 'sadness', 'surprised': 'surprise'})
validate_dataset

,text,true_label,pred_label,probability
0,Then [NAME] insults [NAME] and he cries,neutral,neutral,0.664009
1,Nah they'll make their own post that's really ...,anger,neutral,0.534499
2,The fact that big corporations that made their...,anger,anger,0.558378
3,"Maybe so , i just dont know about him, team ha...",optimistic,surprise,0.510790
4,Thanks for not answering my question. :),happy,happy,0.938847
...,...,...,...,...
495,this would work!!,optimistic,optimistic,0.466005
496,Arguing a player is the GOAT among every posit...,anger,anger,0.856974
497,"I used to have no problems finding them, but I...",sadness,sadness,0.691600
498,"A 70’s Pontiac bonneville , it was a huge boat.",affectionate,affectionate,0.511790


In [19]:
# Export to CSV
validate_dataset_name = 'validate_dataset.csv'
validate_dataset.to_csv(validate_dataset_name, index=False) 

In [20]:
# Upload baseline
baseline_dataset_uri = S3Uploader.upload(f"{validate_dataset_name}", baseline_data_uri)
baseline_dataset_uri

's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

## Setup Baseline

In [13]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

baseline_dataset_uri = 's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

emotions_base_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m4.4xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

# Name of the model quality baseline job
baseline_job_name = f"AIEmotions-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"


# Execute the baseline suggestion job.
job = emotions_base_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="MulticlassClassification",
    inference_attribute="pred_label",
    probability_attribute="probability",
    ground_truth_attribute="true_label",
)
job.wait(logs=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name AIEmotions-model-baseline-job-2024-02-22-0926



Job Name:  AIEmotions-model-baseline-job-2024-02-22-0926
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


ClientError: An error occurred (AccessDeniedException) when calling the CreateProcessingJob operation: User: arn:aws:sts::339712915394:assumed-role/LabRole/SageMaker is not authorized to perform: sagemaker:CreateProcessingJob on resource: arn:aws:sagemaker:us-east-1:339712915394:processing-job/AIEmotions-model-baseline-job-2024-02-22-0926 with an explicit deny in an identity-based policy

In [14]:
baseline_job = emotions_base_model_quality_monitor.latest_baselining_job

In [15]:
multiclass_metrics = baseline_job.baseline_statistics().body_dict["multiclass_classification_metrics"]
pd.json_normalize(multiclass_metrics).T

AttributeError: 'NoneType' object has no attribute 'baseline_statistics'

In [16]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["multiclass_classification_constraints"]).T

AttributeError: 'NoneType' object has no attribute 'suggested_constraints'

## Setup continuous model monitoring

In [17]:
# Generate a small sample from our testing data
synthetic_sample = df_test_new.sample(50)
synthetic_sample = synthetic_sample.reset_index(drop=True)
#synthetic_sample.to_csv('synthetic_data.csv', index=False)

In [18]:
%%time

import numpy as np
import time

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

#flat_list = []
for i in range(50):
    result = predictor.predict({'text': synthetic_sample['text'][i]})
    #flat_list.append(float("%.3f" % (np.array(result))))
    time.sleep(0.5)

print("Done!")
#print("predictions: \t{}".format(np.array(flat_list)))

Sending test traffic to the endpoint AIEmotion-model-quality-monitor-2024-02-22-0923. 
Please wait...
Done!
CPU times: user 139 ms, sys: 9.92 ms, total: 148 ms
Wall time: 38.7 s


In [19]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket_name, Prefix=data_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-1918/AllTraffic/2024/02/21/19/22-11-666-e235b121-fd0c-4af9-8bc9-881988992d84.jsonl
 sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-1918/AllTraffic/2024/02/21/19/23-11-769-244da09e-f334-460a-8f20-575c80bac8e3.jsonl
 sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-1918/AllTraffic/2024/02/21/19/31-04-852-4a6dd6c4-b5bf-4dd4-94a0-8ff1fd6d3720.jsonl
 sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-2043/AllTraffic/2024/02/21/20/47-11-897-3a8e40ca-5eb3-42d2-8759-5d56a8b3f77a.jsonl
 sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-2043/AllTraffic/2024/02/21/20/48-13-428-0cbe3d06-9c08-4dd7-929c-ccb5841cdb74.jsonl
 sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-mo

In [20]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket_name, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"{\"text\": \"Did have this weird situation where it didn\\u2019t round one of the results \"}","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"[\n  \"{\\\"slack_app\\\": {\\\"neutral\\\": 0.3816416263580322, \\\"anger\\\": 0.17415519058704376, \\\"sadness\\\": 0.14958195388317108}, \\\"predictions\\\": \\\"neutral\\\"}\",\n  \"application/json\"\n]","encoding":"JSON"}},"eventMetadata":{"eventId":"b13642c8-7e7f-42c2-8547-e1b7fc3a3fa8","inferenceTime":"2024-02-22T01:56:04Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"{\"text\": \"Did have this weird situation where it didn\\u2019t round one of the results \"}","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"[\n  \"{\\\"slack_app\\\": {\\\"neutral\\\": 0.3816416263

In [21]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "{\"text\": \"Did have this weird situation where it didn\\u2019t round one of the results \"}",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "[\n  \"{\\\"slack_app\\\": {\\\"neutral\\\": 0.3816416263580322, \\\"anger\\\": 0.17415519058704376, \\\"sadness\\\": 0.14958195388317108}, \\\"predictions\\\": \\\"neutral\\\"}\",\n  \"application/json\"\n]",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "b13642c8-7e7f-42c2-8547-e1b7fc3a3fa8",
    "inferenceTime": "2024-02-22T01:56:04Z"
  },
  "eventVersion": "0"
}


In [22]:
# Generating Ground truth
upload_time = datetime.utcnow()
target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
synthetic_sample['emotions'].to_json('output.jsonl', orient='records', lines=True)
S3Uploader.upload_string_as_file_body('output.jsonl', target_s3_uri)

's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-22-09-18-47/2024/02/22/09/3047.jsonl'

In [23]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime
##Monitoring schedule name
model_monitor_schedule_name = (f"AIEmotion-model-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}")

endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        inference_attribute = 'predictions',
        destination="/opt/ml/processing/input/endpoint"
    )

emotions_base_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_monitor_schedule_name,
    endpoint_input=endpoint_input,
    output_s3_uri=s3_report_path,
    constraints=baseline_job.suggested_constraints(),
    ground_truth_input=ground_truth_upload_path,
    problem_type = 'MulticlassClassification',
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

AttributeError: 'NoneType' object has no attribute 'suggested_constraints'

In [ ]:
emotions_base_model_quality_monitor.describe_schedule()

In [ ]:
executions = emotions_base_model_quality_monitor.list_executions()
executions

In [ ]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = emotions_base_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

In [ ]:
while not executions:
    executions = emotions_base_model_quality_monitor.list_executions()
    print(".", end="", flush=True)
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

In [ ]:
latest_execution = emotions_base_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

In [ ]:
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

## Analyze Model Quality

In [ ]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")

namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": model_monitor_schedule_name},
]

In [ ]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

In [ ]:
alarm_name = "MODEL_QUALITY_F1_SCORE"
alarm_desc = (
    "Trigger an CloudWatch alarm when the f1 score drifts away from the baseline constraints"
)
mdoel_quality_f2_drift_threshold = (
    0.6  ##Setting this threshold purposefully low to see the alarm quickly.
)
metric_name = "f1"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_f2_drift_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="breaching",
)

## Release Resources

In [50]:
emotions_base_model_quality_monitor.delete_monitoring_schedule()
sleep(60)

predictor.delete_endpoint()
predictor.delete_model()

INFO:sagemaker:Deleting Monitoring Schedule with name: AIEmotion-model-monitoring-schedule-2024-02-21-2003
INFO:sagemaker.model_monitor.model_monitoring:Deleting Model Quality Job Definition with name: model-quality-job-definition-2024-02-21-20-03-49-232
INFO:sagemaker:Deleting endpoint configuration with name: AIEmotion-model-quality-monitor-2024-02-21-1918


ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "aiemotion-model-quality-monitor-2024-02-21-1918".